# Training your Hand Model
---


This file is used to train your hand model based on the data you have saved for various poses and their samples. This will create a robust model for detecting your hand sign.






### Upload your hand_model.pkl

In [ ]:
from google.colab import files
uploaded = files.upload()

### Install requirements for the colab notebook

In [ ]:
!pip install torch torchvision

import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

### Opening and processing the data from the file

#### Preparing the data

In [ ]:
filename = list(uploaded.keys())[0]
# Open the pickle file correctly
with open(filename, 'rb') as f:
    raw_data = pickle.load(f)

X = []
y = []


for label, samples in raw_data.items():
  for landmark_list in samples:
      X.append(landmark_list)
      y.append(label)

X = np.array(X, dtype=np.float32)
y = np.array(y)

# Encode labels to numeric classes
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


#### Creating a GestureDataset Class for training

In [ ]:
class GestureDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = GestureDataset(X_train, y_train)
val_dataset   = GestureDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

#### Creating a HandGestureClassifier Class for training and loading up the model

In [ ]:
class HandGestureClassifier(nn.Module):
    def __init__(self, num_classes):
        super(HandGestureClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),                  # [B, 21, 2] → [B, 42]
            nn.Linear(42, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = HandGestureClassifier(num_classes=len(le.classes_)).to(device)

### Training the model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = int(input("Enter the number of epochs (default value : 20)  "))
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == targets).sum().item()

    acc = correct / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f} - Acc: {acc*100:.2f}%")

### Evaluating the model over the test cases

In [ ]:
model.eval()
correct = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        correct += (outputs.argmax(1) == targets).sum().item()

val_acc = correct / len(val_dataset)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

Validation Accuracy: 83.33%


## Download the model file

> Training is completed. Now you can download your files from here once you are **finished with above steps** and also happy with the **Validation Accuracy**.

In [ ]:
torch.save(model.state_dict(), "gesture_model.pth")

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

files.download("gesture_model.pth")
files.download("label_encoder.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>